In [52]:
!pip install requests beautifulsoup4 PyPDF2 pdfplumber transformers
import requests
from bs4 import BeautifulSoup
import PyPDF2
import os
import re
import pdfplumber
import csv
from transformers import MarianMTModel, MarianTokenizer

# Adopción y Divorcio: Web Scrapping

In [2]:
url_adopcion: str = "https://consmardelplata.esteri.it/es/servizi-consolari-e-visti/servizi-per-il-cittadino-italiano/altri-servizi/adopciones/"

headers: dict[str, str] = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0 Safari/537.36"
}
response_adopcion: requests = requests.get(url_adopcion, headers=headers)

soup_adopcion: BeautifulSoup = BeautifulSoup(response_adopcion.content, "html.parser")
entry_content_adopcion: str = soup_adopcion.find("div", class_="entry-content")

adopcion: str = entry_content_adopcion.get_text(separator="").strip()

In [3]:
print(adopcion)

Informativa sulla protezione delle persone fisiche con riguardo al trattamento dei dati personali
RECONOCIMIENTO DE SENTENCIAS DE ADOPCION ARGENTINAS EN ITALIA
Los ciudadanos italianos inscriptos en AIRE de un Comune italiano, pueden solicitar el reconocimiento de las sentencias de adopción de sus hijos al Tribunal de Menores italiano competente.
El trámite de reconocimiento no es totalmente automático; la solicitud puede ser denegada por el Tribunal italiano competente por cuestiones tanto de forma como de fondo (quedando abierta la instancia de apelación) y puede demorar un tiempo considerable.
Si la adopción plena se produjo antes de que el adoptado cumpliera 18 años, una vez que se expide favorablemente el Tribunal de Menores italiano, se envía dicho dictamen y el acta de nacimiento del adoptado al Comune de inscripción del adoptante de ciudadanía italiana.
En caso de adopción posterior al cumplimiento de los 18 años del adoptado, o si los adoptantes de ciudadanía italiana hubieren

In [4]:
url_divorcio: str = 'https://conslaplata.esteri.it/es/servizi-consolari-e-visti/servizi-per-il-cittadino-italiano/stato-civile/reconocimiento-de-sentencias-extranjeras/'

response_divorcio: requests = requests.get(url_divorcio, headers=headers)
soup_divorcio: BeautifulSoup = BeautifulSoup(response_divorcio.content, "html.parser")
entry_content_divorcio: str = soup_divorcio.find("div", class_="entry-content")
divorcio: str = entry_content_divorcio.get_text(separator='').strip()

print(divorcio)

Divorcio
Para la validación de la sentencia de divorcio, deberá presentarse la siguiente documentación:

fotocopia legalizada de la sentencia de divorcio.

Para el reconocimiento en Italia de las sentencias de divorcio argentinas, los interesados deben presentar la fotocopia legalizada de la sentencia de divorcio. Las legalizaciones deben efectuarse en el Juzgado donde ha sido expedida la sentencia y en el Ministerio de Relaciones Exteriores, Comercio Internacional y Culto (con la Apostilla de la Convención de La Haya del 5 de octubre de 1961).
El documento debe contener:

todos los elementos esenciales solicitados en una sentencia de divorcio según las leyes italianas (eventuales apelaciones, acuerdos patrimoniales y/o relativos a los hijos menores, notificaciones del inicio del procedimiento, rebeldía, decisión final)
los datos que identifiquen a los interesados (fecha y lugar de nacimiento, domicilio a la fecha del divorcio)
la indicación de que es una sentencia FIRME o CONSENTIDA o

# Circolaire K28, Circolaire K32 y Guía desde PDF
- Lectura de PDF con PyPDF y pdfplumber
- Traducción de leyes Italianas a español.

In [5]:
PATH: str = os.getcwd()

PATH_circolareK28: str = os.path.join(PATH, "CircolareK28.pdf")
PATH_circolare32: str = os.path.join(PATH, "Circolare32.pdf")
PATH_guia: str = os.path.join(PATH,"guia.pdf")

In [6]:
all_pdf: str = [PATH_circolare32, PATH_circolareK28, PATH_guia]

def pdf_to_text(pdf_path: str) -> str:
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text: str = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def pdf_to_text_plumber(pdf_path: str) -> str:
  with pdfplumber.open(PATH_circolare32) as pdf:
      text = ''
      for page in pdf.pages:
          text += page.extract_text()
      return text

circolareK28: str = pdf_to_text(PATH_circolareK28)
circolare32: str = pdf_to_text(PATH_circolare32)
guia: str = pdf_to_text(PATH_guia)

In [28]:
guia_filtrada = guia.replace('\n','')
patron = r"¿TENGO ASCENDENCIA ITALIANA\?.*?EN SACAR CIUDADANÍA ITALIANA, AYUDARTE ES UN PRIVILEGIO\."
guia_filtrada = re.search(patron, guia_filtrada, re.DOTALL)
guia_filtrada = guia_filtrada.group(0)

len(guia_filtrada)

19426

In [53]:
def write_csv(text: str, name) -> None:
    with open(f'{name}.csv', "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(text)

In [55]:
write_csv(guia_filtrada, 'guia')

## Traducción de Italiano a Español

In [46]:
model_name = "Helsinki-NLP/opus-mt-it-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_text(text, model, tokenizer):
    sentences = text.split('\n')
    translated_sentences = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", max_length=512, truncation=True)

        translated = model.generate(**inputs)

        translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
        translated_sentences.append(translated_text)

    full_translation = ' '.join(translated_sentences)
    return full_translation



/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [47]:
circolareK28_traducido = translate_text(circolareK28, model, tokenizer)
circolare32_traducido = translate_text(circolare32, model, tokenizer)

In [48]:
circolare32_traducido

':\' Departamento de Asuntos Internos y Territoriales Dirección Central de Servicios Demográficos 1 3 DE JUNIO DE 2001 Roma, :s CIRCOLAR N°, 2... - A LOS PREFECTOS DE LA REPÚBLICA SUS SIENTOS - AL COMISARIO DEL GOBIERNO PARA LA PROVINCIA DE 39100 8 O L Z A N O - AL COMISARIO DEL GOBIERNO PARA LA PROVINCIA DE 38100 T R E N T O - AL PRESIDENTE DE LA JUNTA REGIONAL DE LA VALLE DE AOSTA 11100 A O S T A y, por conocimiento: - AL COMISARIO DEL ESTADO PARA LA REGIÓN SICILIA 90100 P A L E R M O - AL REPRESENTANTE DEL GOBIERNO PARA LA REGIÓN DE SARDEÑA 09100 C A G L IA R I OBJETO: Legge 28 maggio 2007, n. 68. Supresión del permiso de residencia para el turismo. Inscripción anagráfica de los descendientes de ciudadanos italianos para Nacimiento. En el Diario Oficial no 126, de 10 de junio de 2007, se publicó la Ley. 28 de mayo de 2007, no 68, entrada en vigor el día siguiente, por el que se establece "Disciplina dei estancias de corta duración de extranjeros para visitas, negocios, turismo y est

In [51]:
circolareK28_traducido

'Circulación K.21.1 (1991) La Circulación K.21.1 para los Alcaldes en materia de ciudadanía - No, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no. Circulación no K. 28.1 8 de abril de 1991 del Ministerio del Interior - No, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no. Reconocimiento de la posesión del estatuto civitatis italiano a los ciudadanos extranjeros de cepa italiana - No, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no. Se ha podido constatar que cada vez son más las demandas de En el caso de las ayudas a la construcción naval, las ayudas a la construcción naval y a la mejora de las condiciones de vida y de vida de los trabajadores, así como las ayudas a la construcción naval y a la mejora de las condiciones de vida y de trabajo, así como las ayudas a la construcción naval y a la mejora de las condici

In [57]:
write_csv(circolareK28_traducido, 'circolareK28')
write_csv(circolare32_traducido, 'circolare32')